In [14]:
import os
import csv
import numpy as np

In [15]:
main_dir ="/Users/08volt/Desktop/StremingML/My_results"
old_drifts = ["sea_P(y)","sea_P(y_x)","sea_P(x_y)","sine_P(y)","sine_P(y_x)","sine_P(x_y)"]
new_drifts = ["jitter","disappearing-minority","appearing-minority","appearing-clusters",
             "shapeshift","minority-share","splitting-clusters","clusters-movement","borderline"]
type_drift = ["incremental","sudden","periodic"]
unbalance_lvls = [1,2,3,4]
stats_w = ["Recall_0","Recall_1", "F1_0", "F1_1", "Gmean"]
stats_tot =["time", "memory"]
algs = ["RebalanceStream","CSMOTE","improved_UOB","improved_OOB","WEOB1","WEOB2","ESOS_ELM","UOB","OOB"]

In [16]:
def import_csv(csvfilename):
    data = []
    with open(csvfilename, "r") as scraped:
        reader = csv.reader(scraped, delimiter=',')
        row_index = 0
        for row in reader:
            if row:  # avoid blank lines
                row_index += 1
                columns = [str(row_index)]+ [row[i] for i in range(len(row))]
                data.append(columns)
        #print(f"{row_index} valid rows")
        
    return data

# MEAN ALL ROWS

# OLD

In [17]:
final_dict = {}

for old in old_drifts:
    path = main_dir + "/" + old[old.index("P"):]   #old = sea_P(y)
    if old not in final_dict:
        final_dict[old] = {}
        
    for ds in os.listdir(path):
        #check if ds in old (directory giusta)
        if old[:3] == ds[:3]:
            path1 = path + "/" + ds
            
            unbalance = 0
            for u in unbalance_lvls:
                if str(u) in ds:
                    unbalance = u
            assert unbalance != 0
                        
            if unbalance not in final_dict[old]:
                final_dict[old][unbalance] = {}
                
            dtype = "sudden"
            if "G" in ds:
                dtype = "incremental"
                
            if dtype not in final_dict[old][unbalance]:
                final_dict[old][unbalance][dtype] = {}

            for alg in os.listdir(path1):
                if "DS_Store" in alg:
                    continue
                path2 = path1 + "/" + alg
                class0 = []
                class1 = []
                
                print(f"{old} {unbalance} {dtype} {alg}")
                #prendo i valori di tutti i 10 esperimenti
                for result in os.listdir(path2):
                    if "err" not in result or "DS_Store" in result:
                        continue
                    start = False
                    data = import_csv(path2+"/"+ result)
                    daily0 = []
                    daily1 = []
                    for data_row in data:
                        if data_row[1] == "learning evaluation instances":
                            start = True
                            continue
                        if not start:
                            continue
                        
                        try:
                            daily0.append(float(data_row[11]))
                        except:
                            if data_row[11] == "?":
                                daily0.append(0)
                            else:
                                print("ZERO")
                                print(data_row)
                        try:
                            daily1.append(float(data_row[12]))
                        except:
                            if data_row[12] == "?":
                                daily1.append(0)
                            else:
                                print("ONE")
                                print(data_row)
                    if len(daily0)>0 and len(daily1)>0:
                        class0.append(np.mean(daily0))
                        class1.append(np.mean(daily1))
                            
                        
                    
                assert len(class0) > 0
                assert len(class1) > 0
                assert len(class1) == len(class0)
                if np.mean(class0) != np.mean(class0):
                    print(class0)
                if np.mean(class1) != np.mean(class1):
                    print(class1)
                if alg == "OSHAT":
                    alg = "CSMOTE"
                final_dict[old][unbalance][dtype][alg] = {}
                final_dict[old][unbalance][dtype][alg]["class0"] ={} 
                final_dict[old][unbalance][dtype][alg]["class1"] = {}
                final_dict[old][unbalance][dtype][alg]["class0"]["mean"] = np.mean(class0)
                final_dict[old][unbalance][dtype][alg]["class1"]["mean"] = np.mean(class1)
                final_dict[old][unbalance][dtype][alg]["class0"]["std"] =np.std(class0)
                final_dict[old][unbalance][dtype][alg]["class1"]["std"] = np.std(class1)

                print(np.mean(class0))
                print(np.mean(class1))
                    
                    
                
        

sea_P(y) 1 sudden improved_OOB
98.49474321430235
99.54089343403605
sea_P(y) 1 sudden WEOB1
98.32139889225236
99.62410645869856
sea_P(y) 1 sudden OOB
98.52196764072197
99.37239188613978
sea_P(y) 1 sudden ESOS_ELM
97.36586030796744
74.88531568967352
sea_P(y) 1 sudden improved_UOB
97.99498702056826
99.54715539354268
sea_P(y) 1 sudden WEOB2
97.99498702056826
99.54715539354268
sea_P(y) 1 sudden UOB
98.52196764072197
99.37239188613978
sea_P(y) 1 sudden OSHAT
98.1380466115811
99.1298111232395
sea_P(y) 1 sudden RebalanceStream
98.34111324703679
98.77901438192563
sea_P(y) 3 incremental improved_OOB
92.55927832304303
74.69677962376373
sea_P(y) 3 incremental WEOB1
92.21628075311155
74.95793600540647
sea_P(y) 3 incremental OOB
91.94348531571038
74.85215830768715
sea_P(y) 3 incremental ESOS_ELM
92.38578205575467
57.98461674609191
sea_P(y) 3 incremental improved_UOB
91.67022385246159
74.99641358617649
sea_P(y) 3 incremental WEOB2
91.67022385246159
74.99641358617649
sea_P(y) 3 incremental UOB
91.3001

84.61762471218285
79.16836873893385
sea_P(y_x) 4 sudden WEOB1
84.15631039061452
79.49903689096905
sea_P(y_x) 4 sudden OOB
81.88789207567439
83.49906834894801
sea_P(y_x) 4 sudden ESOS_ELM
86.61376944469342
59.51475809676761
sea_P(y_x) 4 sudden improved_UOB
83.05381752155596
79.52648808511883
sea_P(y_x) 4 sudden WEOB2
83.05381752155596
79.52648808511883
sea_P(y_x) 4 sudden UOB
79.33607665957877
83.21801421620697
sea_P(y_x) 4 sudden OSHAT
83.03231979008375
79.85996340688904
sea_P(y_x) 4 sudden RebalanceStream
83.9311887128514
77.61909238918342
sea_P(x_y) 1 sudden improved_OOB
92.86406686942492
73.1594464099324
sea_P(x_y) 1 sudden WEOB1
88.71349402159969
77.88871157839172
sea_P(x_y) 1 sudden OOB
91.1786580210117
75.727128095837
sea_P(x_y) 1 sudden ESOS_ELM
90.20532997343079
60.421071872899574
sea_P(x_y) 1 sudden improved_UOB
80.69260830480997
80.5652049255669
sea_P(x_y) 1 sudden WEOB2
80.69260830480997
80.5652049255669
sea_P(x_y) 1 sudden UOB
79.05786808927414
81.0271105619426
sea_P(x_y) 1

99.31406741006654
98.60433977674
sine_P(y) 2 incremental WEOB1
99.14198412634696
98.85630357365669
sine_P(y) 2 incremental OOB
99.06072593378332
98.85774844014806
sine_P(y) 2 incremental ESOS_ELM
97.79254352603357
75.60011975215147
sine_P(y) 2 incremental improved_UOB
98.37198320094151
98.68775722879613
sine_P(y) 2 incremental WEOB2
98.37198320094151
98.68775722879613
sine_P(y) 2 incremental UOB
97.3765027453791
98.88043294367348
sine_P(y) 2 incremental OSHAT
96.77760927760423
98.02363422151201
sine_P(y) 2 incremental RebalanceStream
96.27044040852708
94.85890117392951
sine_P(y) 1 incremental improved_OOB
99.4633635125738
97.50287498794246
sine_P(y) 1 incremental WEOB1
99.36404333262051
97.84738351535154
sine_P(y) 1 incremental OOB
99.34924477949436
97.64358258608543
sine_P(y) 1 incremental ESOS_ELM
96.93695435247844
75.65654654556832
sine_P(y) 1 incremental improved_UOB
98.0755814184898
97.6335123165101
sine_P(y) 1 incremental WEOB2
98.0755814184898
97.6335123165101
sine_P(y) 1 increm

98.69808892379018
99.1710313638406
sine_P(x_y) 4 sudden WEOB2
98.69808892379018
99.1710313638406
sine_P(x_y) 4 sudden UOB
98.46788803582086
99.11592736565454
sine_P(x_y) 4 sudden OSHAT
98.46849097025105
98.8906496349779
sine_P(x_y) 4 sudden RebalanceStream
98.24252009642875
98.53959974326095
sine_P(x_y) 1 sudden improved_OOB
99.56734194992428
96.80188940195538
sine_P(x_y) 1 sudden WEOB1
99.40088977066856
97.41902750729074
sine_P(x_y) 1 sudden OOB
99.5116124945969
96.97004098872458
sine_P(x_y) 1 sudden ESOS_ELM
97.96094965627675
71.82848688392866
sine_P(x_y) 1 sudden improved_UOB
97.85336454624239
97.90304621600185
sine_P(x_y) 1 sudden WEOB2
97.85336454624239
97.90304621600185
sine_P(x_y) 1 sudden UOB
97.34344892089942
98.09494887625432
sine_P(x_y) 1 sudden OSHAT
96.9225173621081
91.26211321782476
sine_P(x_y) 1 sudden RebalanceStream
99.76631678685126
91.03442952492216
sine_P(x_y) 2 incremental improved_OOB
99.52048455537349
98.35135557642502
sine_P(x_y) 2 incremental WEOB1
99.327566534

In [18]:
old = final_dict.copy()

# NEW

In [19]:
final_dict = old.copy()

In [20]:
for new in new_drifts:
    path = main_dir + "/" + new 
    
    if new not in final_dict:
        final_dict[new] = {}
        
    for unbalance in range(1,5):
        if unbalance not in final_dict[new]:
            final_dict[new][unbalance] = {}


        path1 = path + "/" + str(unbalance)

        for alg in os.listdir(path1):
            if "DS_Store" in alg:
                continue
            path2 = path1 + "/" + alg
            
            alg_dtypes = {}
            alg_dtypes["incremental"] = {}
            alg_dtypes["sudden"] = {}
            alg_dtypes["periodic"] = {}
            alg_dtypes["incremental"]["class0"] = []
            alg_dtypes["sudden"]["class0"] = []
            alg_dtypes["periodic"]["class0"] = []
            alg_dtypes["incremental"]["class1"] = []
            alg_dtypes["sudden"]["class1"] = []
            alg_dtypes["periodic"]["class1"] = []
            
            
            print(f"{new} {unbalance} {alg}")
            #prendo i valori di tutti i 10 esperimenti dei 3 dtype
            
            for result in os.listdir(path2):
                if "err" not in result or "DS_Store" in result:
                    continue
                daily0 = []
                daily1 = []
                data = import_csv(path2+"/"+ result)
                start = False
                
                for data_row in data:
                    if data_row[1] == "learning evaluation instances":
                        start = True
                        continue
                    if not start:
                        continue

                    try:
                        daily0.append(float(data_row[11]))
                    except:
                        if data_row[11] == "?":
                            daily0.append(0.0)
                        else:
                            print("ZERO")
                            print(data_row)
                    try:
                        daily1.append(float(data_row[12]))
                    except:
                        if data_row[12] == "?":
                            daily1.append(0.0)
                        else:
                            print("ONE")
                            print(data_row)
                        
                    
                
                try:
                    if len(daily0)>0 and len(daily1)>0:
                        if alg == "ESOS_ELM" and new == "appearing-minority":
                            print(daily0)
                            print(daily1)
                        alg_dtypes[result[:result.index("_")]]["class0"].append(np.mean(daily0))
                        alg_dtypes[result[:result.index("_")]]["class1"].append(np.mean(daily1))
                except:
                    print(f"error with {result}")
            for k,i in alg_dtypes.items():      
                if len(i["class0"]) < 10:
                    print(f"N EXP {len(i['class0'])} in 0")
                if len(i["class1"]) < 10:
                    print(f"N EXP {len(i['class1'])} in 1")
                if k not in final_dict[new][unbalance]:
                    final_dict[new][unbalance][k] = {}
                assert alg not in final_dict[new][unbalance][k]
                final_dict[new][unbalance][k][alg] = {}
                final_dict[new][unbalance][k][alg]["class0"] ={} 
                final_dict[new][unbalance][k][alg]["class1"] = {}
                final_dict[new][unbalance][k][alg]["class0"]["mean"] = np.mean(i["class0"])
                final_dict[new][unbalance][k][alg]["class1"]["mean"] = np.mean(i["class1"])
                final_dict[new][unbalance][k][alg]["class0"]["std"] =np.std(i["class0"])
                final_dict[new][unbalance][k][alg]["class1"]["std"] = np.std(i["class1"])
                print(k)
                print(np.mean(i["class1"]))
                print(np.mean(i["class0"]))

jitter 1 improved_OOB
incremental
92.06566158095019
90.00827059471506
sudden
89.01134348388123
89.75685529663838
periodic
92.06566158095019
90.00827059471506
jitter 1 WEOB1
incremental
94.99333218617934
87.61120235618056
sudden
91.35411359554068
87.59248756665909
periodic
94.99333218617934
87.61120235618054
jitter 1 CSMOTE
incremental
82.28000932164011
77.14046903044438
sudden
90.08815293429414
79.46468104947641
periodic
82.28000932164011
77.14046903044438
jitter 1 OOB
incremental
94.13746666427429
89.24238604329473
sudden
91.50196341289447
88.83034834000122
periodic
94.13746666427429
89.24238604329474
jitter 1 ESOS_ELM
incremental
71.75083386190059
76.02777557779979
sudden
72.87731949004618
90.16423306275671
periodic
70.65448726440661
75.77752556912996
jitter 1 improved_UOB
incremental
95.3882021783071
80.51268204795778
sudden
89.34745798626419
81.10938923649805
periodic
95.3882021783071
80.51268204795778
jitter 1 WEOB2
incremental
95.3882021783071
80.51268204795778
sudden
89.34745798

incremental
95.77399237789416
79.30687829637078
sudden
89.33216672949337
80.03155583221434
periodic
95.77399237789416
79.3068782963708
disappearing-minority 2 RebalanceStream
incremental
41.18348831006883
98.92255664976565
sudden
38.68742411725184
98.79065272054751
periodic
41.183488310068824
98.92255664976565
disappearing-minority 3 improved_OOB
incremental
92.06566158095019
90.00827059471506
sudden
89.01134348388123
89.75685529663838
periodic
92.06566158095019
90.00827059471506
disappearing-minority 3 WEOB1
incremental
94.99333218617934
87.61120235618056
sudden
91.35411359554068
87.59248756665909
periodic
94.99333218617934
87.61120235618054
disappearing-minority 3 CSMOTE
incremental
51.69099700312985
87.80186778504482
sudden
47.739836611935914
88.07301689518354
periodic
94.05434587216095
79.87240931203141
disappearing-minority 3 OOB
incremental
94.13746666427429
89.24238604329473
sudden
91.50196341289447
88.83034834000122
periodic
94.13746666427429
89.24238604329474
disappearing-mino

incremental
95.3882021783071
80.51268204795778
sudden
89.34745798626419
81.10938923649805
periodic
95.3882021783071
80.51268204795778
appearing-minority 1 UOB
incremental
95.77399237789416
79.30687829637078
sudden
89.33216672949337
80.03155583221434
periodic
95.77399237789416
79.3068782963708
appearing-minority 1 RebalanceStream
incremental
41.18348831006883
98.92255664976565
sudden
38.68742411725184
98.79065272054751
periodic
41.183488310068824
98.92255664976565
appearing-minority 2 improved_OOB
incremental
92.06566158095019
90.00827059471506
sudden
89.01134348388123
89.75685529663838
periodic
92.06566158095019
90.00827059471506
appearing-minority 2 WEOB1
incremental
94.99333218617934
87.61120235618056
sudden
91.35411359554068
87.59248756665909
periodic
94.99333218617934
87.61120235618054
appearing-minority 2 CSMOTE
incremental
45.80367838129287
90.96511594973137
sudden
45.337514356373234
89.11375761315979
periodic
7.186540372109827
99.15454633917116
appearing-minority 2 OOB
increment

incremental
95.3882021783071
80.51268204795778
sudden
89.34745798626419
81.10938923649805
periodic
95.3882021783071
80.51268204795778
appearing-minority 2 UOB
incremental
95.77399237789416
79.30687829637078
sudden
89.33216672949337
80.03155583221434
periodic
95.77399237789416
79.3068782963708
appearing-minority 2 RebalanceStream
incremental
41.18348831006883
98.92255664976565
sudden
38.68742411725184
98.79065272054751
periodic
41.183488310068824
98.92255664976565
appearing-minority 3 improved_OOB
incremental
92.06566158095019
90.00827059471506
sudden
89.01134348388123
89.75685529663838
periodic
92.06566158095019
90.00827059471506
appearing-minority 3 WEOB1
incremental
94.99333218617934
87.61120235618056
sudden
91.35411359554068
87.59248756665909
periodic
94.99333218617934
87.61120235618054
appearing-minority 3 CSMOTE
incremental
49.81715187186951
90.42281636366758
sudden
47.099114853735735
90.40593678753113
periodic
7.487574333504087
98.02436638693874
appearing-minority 3 OOB
increment

incremental
95.77399237789416
79.30687829637078
sudden
89.33216672949337
80.03155583221434
periodic
95.77399237789416
79.3068782963708
appearing-minority 3 RebalanceStream
incremental
41.18348831006883
98.92255664976565
sudden
38.68742411725184
98.79065272054751
periodic
41.183488310068824
98.92255664976565
appearing-minority 4 improved_OOB
incremental
92.06566158095019
90.00827059471506
sudden
89.01134348388123
89.75685529663838
periodic
92.06566158095019
90.00827059471506
appearing-minority 4 WEOB1
incremental
94.99333218617934
87.61120235618056
sudden
91.35411359554068
87.59248756665909
periodic
94.99333218617934
87.61120235618054
appearing-minority 4 CSMOTE
incremental
51.368439353304936
88.90531174160651
sudden
46.87622655150996
89.8766024108107
periodic
7.840585153511308
98.53505979873057
appearing-minority 4 OOB
incremental
94.13746666427429
89.24238604329473
sudden
91.50196341289447
88.83034834000122
periodic
94.13746666427429
89.24238604329474
appearing-minority 4 ESOS_ELM
[10

incremental
41.18348831006883
98.92255664976565
sudden
38.68742411725184
98.79065272054751
periodic
41.183488310068824
98.92255664976565
appearing-clusters 1 improved_OOB
incremental
92.06566158095019
90.00827059471506
sudden
89.01134348388123
89.75685529663838
periodic
92.06566158095019
90.00827059471506
appearing-clusters 1 WEOB1
incremental
94.99333218617934
87.61120235618056
sudden
91.35411359554068
87.59248756665909
periodic
94.99333218617934
87.61120235618054
appearing-clusters 1 CSMOTE
incremental
83.41925499644637
92.09555679914398
sudden
83.09429435614018
92.61397438258577
periodic
94.34166131724001
93.87759888729735
appearing-clusters 1 OOB
incremental
94.13746666427429
89.24238604329473
sudden
91.50196341289447
88.83034834000122
periodic
94.13746666427429
89.24238604329474
appearing-clusters 1 ESOS_ELM
incremental
71.23535981211454
79.39439486379386
sudden
70.20734612701176
79.5557568880689
periodic
76.49615971731848
81.82368670377413
appearing-clusters 1 improved_UOB
increm

incremental
94.99333218617934
87.61120235618056
sudden
91.35411359554068
87.59248756665909
periodic
94.99333218617934
87.61120235618054
shapeshift 3 CSMOTE
incremental
93.57722267509442
78.45020519994073
sudden
93.26233002997033
78.70578442427163
periodic
94.8721079203165
76.7728929064513
shapeshift 3 OOB
incremental
94.13746666427429
89.24238604329473
sudden
91.50196341289447
88.83034834000122
periodic
94.13746666427429
89.24238604329474
shapeshift 3 ESOS_ELM
incremental
71.69591432894627
78.45630581052372
sudden
71.00208190310312
78.62449547573722
periodic
68.73153076434686
77.73608536404643
shapeshift 3 improved_UOB
incremental
95.3882021783071
80.51268204795778
sudden
89.34745798626419
81.10938923649805
periodic
95.3882021783071
80.51268204795778
shapeshift 3 WEOB2
incremental
95.3882021783071
80.51268204795778
sudden
89.34745798626419
81.10938923649805
periodic
95.3882021783071
80.51268204795778
shapeshift 3 UOB
incremental
95.77399237789416
79.30687829637078
sudden
89.33216672949

incremental
92.06566158095019
90.00827059471506
sudden
89.01134348388123
89.75685529663838
periodic
92.06566158095019
90.00827059471506
splitting-clusters 1 WEOB1
incremental
94.99333218617934
87.61120235618056
sudden
91.35411359554068
87.59248756665909
periodic
94.99333218617934
87.61120235618054
splitting-clusters 1 CSMOTE
incremental
78.77373965951607
88.52005159189966
sudden
79.0306366916925
89.52050085507766
periodic
89.69359545046407
90.95400727120214
splitting-clusters 1 OOB
incremental
94.13746666427429
89.24238604329473
sudden
91.50196341289447
88.83034834000122
periodic
94.13746666427429
89.24238604329474
splitting-clusters 1 ESOS_ELM
incremental
72.35406354630332
80.14418151224334
sudden
68.6921232417563
79.83181106474575
periodic
75.81629950541449
82.58510775080228
splitting-clusters 1 improved_UOB
incremental
95.3882021783071
80.51268204795778
sudden
89.34745798626419
81.10938923649805
periodic
95.3882021783071
80.51268204795778
splitting-clusters 1 WEOB2
incremental
95.38

incremental
95.3882021783071
80.51268204795778
sudden
89.34745798626419
81.10938923649805
periodic
95.3882021783071
80.51268204795778
clusters-movement 2 UOB
incremental
95.77399237789416
79.30687829637078
sudden
89.33216672949337
80.03155583221434
periodic
95.77399237789416
79.3068782963708
clusters-movement 2 RebalanceStream
incremental
41.18348831006883
98.92255664976565
sudden
38.68742411725184
98.79065272054751
periodic
41.183488310068824
98.92255664976565
clusters-movement 3 improved_OOB
incremental
92.06566158095019
90.00827059471506
sudden
89.01134348388123
89.75685529663838
periodic
92.06566158095019
90.00827059471506
clusters-movement 3 WEOB1
incremental
94.99333218617934
87.61120235618056
sudden
91.35411359554068
87.59248756665909
periodic
94.99333218617934
87.61120235618054
clusters-movement 3 CSMOTE
incremental
87.50439961194226
68.22377447806574
sudden
81.21791122224923
72.54503029563304
periodic
92.75996810803476
78.14273263095482
clusters-movement 3 OOB
incremental
94.1

incremental
41.18348831006883
98.92255664976565
sudden
38.68742411725184
98.79065272054751
periodic
41.183488310068824
98.92255664976565


In [21]:
final_dict

{'sea_P(y)': {1: {'sudden': {'improved_OOB': {'class0': {'mean': 98.49474321430235,
      'std': 0.08252391356674567},
     'class1': {'mean': 99.54089343403605, 'std': 0.054668081390178994}},
    'WEOB1': {'class0': {'mean': 98.32139889225236,
      'std': 0.052809029231749176},
     'class1': {'mean': 99.62410645869856, 'std': 0.047344734800184494}},
    'OOB': {'class0': {'mean': 98.52196764072197, 'std': 0.06410014024085092},
     'class1': {'mean': 99.37239188613978, 'std': 0.09008234479071989}},
    'ESOS_ELM': {'class0': {'mean': 97.36586030796744,
      'std': 0.19451966631874398},
     'class1': {'mean': 74.88531568967352, 'std': 0.34930759356982527}},
    'improved_UOB': {'class0': {'mean': 97.99498702056826,
      'std': 0.11013341750523027},
     'class1': {'mean': 99.54715539354268, 'std': 0.08179884078971333}},
    'WEOB2': {'class0': {'mean': 97.99498702056826,
      'std': 0.11013341750523027},
     'class1': {'mean': 99.54715539354268, 'std': 0.08179884078971333}},
   

In [22]:
# def stats_of(_dict ):
#     values_0 = []
#     values_1 = []
#     for alg, values in _dict.items():
#         if "ELM" in alg:
#             continue
#         values_0.append(values["class0"])
#         values_1.append(values["class1"])
#     return np.mean(values_0),np.mean(values_1), np.max(values_0), np.max(values_1), np.min(values_0), np.min(values_1)


In [23]:

with open('Recall_withSTD.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["drift","unbalance","type","alg","Recall_0", "Recall_1","Recall_0_std", "Recall_1_std"])
    for drift, dicts in final_dict.items():
        for unbalance, dtypes in dicts.items():
            for dtype, algs in dtypes.items():
#                 mean0, mean1,max0,max1,min0,min1 = stats_of(algs)
                for alg, values in algs.items():

                    print([drift,unbalance,dtype,alg])
                    print(values)
                    writer.writerow([drift,unbalance,dtype,alg,values["class0"]["mean"],values["class1"]["mean"],values["class0"]["std"],values["class1"]["std"]])

['sea_P(y)', 1, 'sudden', 'improved_OOB']
{'class0': {'mean': 98.49474321430235, 'std': 0.08252391356674567}, 'class1': {'mean': 99.54089343403605, 'std': 0.054668081390178994}}
['sea_P(y)', 1, 'sudden', 'WEOB1']
{'class0': {'mean': 98.32139889225236, 'std': 0.052809029231749176}, 'class1': {'mean': 99.62410645869856, 'std': 0.047344734800184494}}
['sea_P(y)', 1, 'sudden', 'OOB']
{'class0': {'mean': 98.52196764072197, 'std': 0.06410014024085092}, 'class1': {'mean': 99.37239188613978, 'std': 0.09008234479071989}}
['sea_P(y)', 1, 'sudden', 'ESOS_ELM']
{'class0': {'mean': 97.36586030796744, 'std': 0.19451966631874398}, 'class1': {'mean': 74.88531568967352, 'std': 0.34930759356982527}}
['sea_P(y)', 1, 'sudden', 'improved_UOB']
{'class0': {'mean': 97.99498702056826, 'std': 0.11013341750523027}, 'class1': {'mean': 99.54715539354268, 'std': 0.08179884078971333}}
['sea_P(y)', 1, 'sudden', 'WEOB2']
{'class0': {'mean': 97.99498702056826, 'std': 0.11013341750523027}, 'class1': {'mean': 99.5471553

{'class0': {'mean': 80.51268204795778, 'std': 2.1004446905257126}, 'class1': {'mean': 95.3882021783071, 'std': 1.1817824733161488}}
['jitter', 4, 'periodic', 'WEOB2']
{'class0': {'mean': 80.51268204795778, 'std': 2.1004446905257126}, 'class1': {'mean': 95.3882021783071, 'std': 1.1817824733161488}}
['jitter', 4, 'periodic', 'UOB']
{'class0': {'mean': 79.3068782963708, 'std': 2.5786663683234474}, 'class1': {'mean': 95.77399237789416, 'std': 1.0672154576536783}}
['jitter', 4, 'periodic', 'RebalanceStream']
{'class0': {'mean': 98.92255664976565, 'std': 0.1145581729821165}, 'class1': {'mean': 41.183488310068824, 'std': 6.601762099386483}}
['disappearing-minority', 1, 'incremental', 'improved_OOB']
{'class0': {'mean': 90.00827059471506, 'std': 0.9254519983852814}, 'class1': {'mean': 92.06566158095019, 'std': 1.9458150974979587}}
['disappearing-minority', 1, 'incremental', 'WEOB1']
{'class0': {'mean': 87.61120235618056, 'std': 1.1360127535967521}, 'class1': {'mean': 94.99333218617934, 'std': 

['shapeshift', 1, 'sudden', 'improved_UOB']
{'class0': {'mean': 81.10938923649805, 'std': 1.9109561949523772}, 'class1': {'mean': 89.34745798626419, 'std': 1.2223390628873918}}
['shapeshift', 1, 'sudden', 'WEOB2']
{'class0': {'mean': 81.10938923649805, 'std': 1.9109561949523772}, 'class1': {'mean': 89.34745798626419, 'std': 1.2223390628873918}}
['shapeshift', 1, 'sudden', 'UOB']
{'class0': {'mean': 80.03155583221434, 'std': 2.36300634986352}, 'class1': {'mean': 89.33216672949337, 'std': 1.0974494057971713}}
['shapeshift', 1, 'sudden', 'RebalanceStream']
{'class0': {'mean': 98.79065272054751, 'std': 0.16925550993349445}, 'class1': {'mean': 38.68742411725184, 'std': 6.575675341883652}}
['shapeshift', 1, 'periodic', 'improved_OOB']
{'class0': {'mean': 90.00827059471506, 'std': 0.9254519983852814}, 'class1': {'mean': 92.06566158095019, 'std': 1.9458150974979584}}
['shapeshift', 1, 'periodic', 'WEOB1']
{'class0': {'mean': 87.61120235618054, 'std': 1.1360127535967521}, 'class1': {'mean': 94.

['minority-share', 3, 'sudden', 'WEOB1']
{'class0': {'mean': 87.59248756665909, 'std': 1.176959512570119}, 'class1': {'mean': 91.35411359554068, 'std': 1.8614946090394062}}
['minority-share', 3, 'sudden', 'CSMOTE']
{'class0': {'mean': 80.2511798614041, 'std': 0.0}, 'class1': {'mean': 96.81671887487845, 'std': 0.0}}
['minority-share', 3, 'sudden', 'OOB']
{'class0': {'mean': 88.83034834000122, 'std': 1.0632575465470198}, 'class1': {'mean': 91.50196341289447, 'std': 1.3069327488825373}}
['minority-share', 3, 'sudden', 'ESOS_ELM']
{'class0': {'mean': 79.7662873882407, 'std': 0.16726797092362763}, 'class1': {'mean': 69.85851412279013, 'std': 0.6215226970821395}}
['minority-share', 3, 'sudden', 'improved_UOB']
{'class0': {'mean': 81.10938923649805, 'std': 1.9109561949523772}, 'class1': {'mean': 89.34745798626419, 'std': 1.2223390628873918}}
['minority-share', 3, 'sudden', 'WEOB2']
{'class0': {'mean': 81.10938923649805, 'std': 1.9109561949523772}, 'class1': {'mean': 89.34745798626419, 'std': 

# REALS

In [10]:
real_dict = {}
real_dict["KDDCup"] = {}
real_dict["PAKDD"] = {}
real_dict["Elec"] = {}

for i in real_dict.keys():
    path = f"/Users/08volt/Desktop/StremingML/TEST/Results_union/Real/{i}"
    for alg in os.listdir(path):
        class0 = []
        if "DS_Store" in alg or "ipynb" in alg:
            continue
        class1 = []
        if alg not in real_dict[i]:
            real_dict[i][alg] = {}
        
            
        for exp in os.listdir(path + "/" + alg): 
            if "err" not in exp or "DS_Store" in exp:
                continue
                
            data = import_csv(path + "/" + alg + "/"+ exp)
            daily0 = []
            daily1 = []
            start = False

            for data_row in data:
                if data_row[1] == "learning evaluation instances":
                    start = True
                    continue
                if not start:
                    continue

                try:
                    daily0.append(float(data_row[11]))
                except:
                    if data_row[11] == "?":
                        daily0.append(0.0)
                    else:
                        print("ZERO")
                        print(data_row)
                try:
                    daily1.append(float(data_row[12]))
                except:
                    if data_row[12] == "?":
                        daily1.append(0.0)
                    else:
                        print("ONE")
                        print(data_row)
            try:
                if len(daily0)>0 and len(daily1)>0:
                    class0.append(np.mean(daily0))
                    class1.append(np.mean(daily1))
            except:
                print(f"error with {result}")
        real_dict[i][alg]["class0"] = {}
        real_dict[i][alg]["class1"] = {}
        real_dict[i][alg]["class0"]["mean"] = np.mean(class0)
        real_dict[i][alg]["class1"]["mean"] = np.mean(class1)
        real_dict[i][alg]["class0"]["std"] = np.std(class0)
        real_dict[i][alg]["class1"]["std"] = np.std(class1)
        
                
                

In [11]:
real_dict

{'KDDCup': {'improved_OOB': {'class0': {'mean': 97.15655556275475, 'std': 0.0},
   'class1': {'mean': 99.74249148061718, 'std': 1.4210854715202004e-14}},
  'WEOB1': {'class0': {'mean': 98.88129505568767, 'std': 0.0},
   'class1': {'mean': 99.10139585088145, 'std': 0.0}},
  'OOB': {'class0': {'mean': 96.9798926629041, 'std': 1.4210854715202004e-14},
   'class1': {'mean': 99.86829041088046, 'std': 1.4210854715202004e-14}},
  'ESOS_ELM': {'class0': {'mean': 100.0, 'std': 0.0},
   'class1': {'mean': 0.0, 'std': 0.0}},
  'improved_UOB': {'class0': {'mean': 98.4018521449992,
    'std': 1.4210854715202004e-14},
   'class1': {'mean': 96.02451645343186, 'std': 0.0}},
  'WEOB2': {'class0': {'mean': 98.21314448463167, 'std': 0.0},
   'class1': {'mean': 95.27573057484446, 'std': 1.4210854715202004e-14}},
  'UOB': {'class0': {'mean': 95.94248111874364, 'std': 1.4210854715202004e-14},
   'class1': {'mean': 99.42185439987873, 'std': 0.0}},
  'OSHAT': {'class0': {'mean': 97.40330962335665,
    'std': 

In [13]:
with open('RecallXRecall_real_withSTD.csv', 'w') as csv_file:   
    writer = csv.writer(csv_file)
    writer.writerow(["ds","alg","Recall_0", "Recall_1","Recall_0_std", "Recall_1_std"])
    for ds, dicts in real_dict.items():
        for alg, values in dicts.items():

            if alg == "OSHAT":
                alg = "CSMOTE"
            writer.writerow([ds,alg,values["class0"]["mean"],values["class1"]["mean"],values["class0"]["std"],values["class1"]["std"]])
